In [4]:
################################################################################################################################################################################
# Importamos las librerías necesarias y generamos conexión con Binance para descargar la lista de pares que binance maneja.
################################################################################################################################################################################
import pandas as pd
import os
import numpy as np
from datetime import datetime
import sympy as sp
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from binance import Client

client = Client()

##################################################
# Lista de parámetros.
##################################################
# Parámetros de interpolación.
###################################
smooth_interval = 60
smooth_exp = 3
###################################
# Parámetros de estrategia.
###################################
rango = 20
std_mult = 150
pdi_mult = 25
sdi_mult = 250
z_aprox = 0.01

recent_data = pd.read_csv('../../csvs/streaming/BTCUSDT.csv',skiprows=29300)
recent_data.columns = ['Tiempo','Par','Precio']
recent_data['Tiempo'] = recent_data.Tiempo.astype('datetime64')
recent_data = recent_data.set_index('Tiempo')
recent_data['Precio'] = recent_data.Precio.astype(float)
recent_data = recent_data.reset_index()

pmax_idx = 0
pmin_idx = 0
last_time_buy = recent_data.Tiempo[0]
last_time_sell = recent_data.Tiempo[0]

df_logs = pd.DataFrame(columns=['Tiempo','Accion'])

while True:
    max_points_idx = []
    min_points_idx = []

    ##################################################################################################################################
    # Descarga y limpieza de datos de 5 días en adelante
    ##################################################################################################################################
    recent_data = pd.read_csv('../../csvs/streaming/BTCUSDT.csv',skiprows=40200)
    recent_data.columns = ['Tiempo','Par','Precio']
    recent_data['Tiempo'] = recent_data.Tiempo.astype('datetime64')
    recent_data = recent_data.set_index('Tiempo')
    recent_data['Precio'] = recent_data.Precio.astype(float)
    recent_data = recent_data.reset_index()

    ##################################################
    # Interpolación (suavizado)  del Precio (OPEN) con SciPy y obtención de derivadas con su respectiva interpolación.
    ##################################################
    recent_data['Interpolacion'] = sc.signal.savgol_filter(recent_data.Precio,smooth_interval,smooth_exp)
    recent_data['Primer_Derivada'] = recent_data.Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['PD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Primer_Derivada,smooth_interval,smooth_exp)
    recent_data['Segunda_Derivada'] = recent_data.PD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['SD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Segunda_Derivada,smooth_interval,smooth_exp)
    recent_data['Tercer_Derivada'] = recent_data.SD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['TD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Tercer_Derivada,smooth_interval,3)

    ######################################################################
    # Agregamos el código necesario para encontrar los puntos exactos mínimos y máximos
    ######################################################################
    try:
        initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[60*2:60*2 + 1].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() + rango
    except:
        initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[60*2:60*2 + 1].item()),'%Y-%m-%d %H:%M:%S')].index.item() + rango
    try:
        final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[-1:].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() - rango
    except:
        final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[-1:].item()),'%Y-%m-%d %H:%M:%S')].index.item() - rango

    min_max_sd = pd.DataFrame(columns=['Derivada'])
    lista_min_max = recent_data[initial_date_index:final_date_index][(recent_data[initial_date_index:final_date_index]['TD_Interpolacion'] >= -z_aprox) & (recent_data[initial_date_index:final_date_index]['TD_Interpolacion'] <= z_aprox)]
    indices_min_max = lista_min_max.index.tolist()
    mp = recent_data.Precio[initial_date_index:final_date_index].mean()

    pmin = 0
    pmax = 0

    for PMSD in indices_min_max:
        if PMSD > len(recent_data) - rango:
            break
        if (recent_data.SD_Interpolacion[PMSD:PMSD+1].item() > 0) & (recent_data.PD_Interpolacion[PMSD:PMSD+1].item() > 0):
            pmax_idx = PMSD
            if ((mp + recent_data.SD_Interpolacion[PMSD]*sdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD]*pdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)):
                pmax = recent_data.SD_Interpolacion[PMSD]
            for i in range(rango+1):
                try:
                    # if (i > PMSD) or (PMSD+i >= indices_min_max[-1:][0]):
                    #     break
                    if ((recent_data.SD_Interpolacion[PMSD-i] > pmax)) & ((mp + recent_data.SD_Interpolacion[PMSD-i]*sdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmax_idx = PMSD-i
                    elif ((recent_data.SD_Interpolacion[PMSD+i] > pmax)) & ((mp + recent_data.SD_Interpolacion[PMSD+i]*sdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD+i]*pdi_mult) > (mp + np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmax_idx = PMSD+i
                except:
                    pass
                try:
                    max_points_idx.append(pmax_idx)
                except:
                    pass
        elif (recent_data.SD_Interpolacion[PMSD:PMSD+1].item() < 0) & (recent_data.PD_Interpolacion[PMSD:PMSD+1].item() < 0):
            pmin_idx = PMSD
            if ((mp + recent_data.SD_Interpolacion[PMSD]*sdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD]*pdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)):
                pmin = recent_data.SD_Interpolacion[PMSD]
            for i in range(rango+1):
                try:
                    # if (i > PMSD) or (PMSD+i >= indices_min_max[-1:][0]):
                    #     break
                    if ((recent_data.SD_Interpolacion[PMSD-i] < pmin)) & ((mp + recent_data.SD_Interpolacion[PMSD-i]*sdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmin_idx = PMSD-i
                    elif ((recent_data.SD_Interpolacion[PMSD+i] < pmin)) & ((mp + recent_data.SD_Interpolacion[PMSD+i]*sdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)) & ((mp + recent_data.PD_Interpolacion[PMSD+i]*pdi_mult) < (mp - np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmin_idx = PMSD+i
                except:
                    pass
                try:
                    min_points_idx.append(pmin_idx)
                except:
                    pass
    max_points_idx = list(set(max_points_idx))
    min_points_idx = list(set(min_points_idx))
    max_points_idx.sort()
    min_points_idx.sort()
    
    try:
        if (recent_data.Tiempo[max_points_idx[-1]] != last_time_buy) & (recent_data.Tiempo[max_points_idx[-1]] > last_time_buy) & (recent_data.Tiempo[max_points_idx[-1]] > last_time_sell):
            last_time_buy = recent_data.Tiempo[max_points_idx[-1]]
            print(f'Precio de compra en: {recent_data.Precio[max_points_idx[-1]]}. Tiempo de compra: {recent_data.Tiempo[max_points_idx[-1]]}. Tiempo real de la señal: {datetime.utcnow()}. Diferencia entre señal y compra: {(datetime.utcnow() - recent_data.Tiempo[max_points_idx[-1]]).seconds} segundos.')
            lista_logs = [datetime.utcnow(),'C']
            df_logs.loc[len(df_logs)] = lista_logs
            print(max_points_idx[-1])
    except:
        pass
    try:
        if (recent_data.Tiempo[min_points_idx[-1]] != last_time_sell) & (recent_data.Tiempo[min_points_idx[-1]] > last_time_sell) & (recent_data.Tiempo[min_points_idx[-1]] > last_time_buy):
            last_time_sell = recent_data.Tiempo[min_points_idx[-1]]
            print(f'Precio de venta en: {recent_data.Precio[min_points_idx[-1]]}. Tiempo de venta: {recent_data.Tiempo[min_points_idx[-1]]}. Tiempo real de la señal: {datetime.utcnow()}. Diferencia entre señal y compra: {(datetime.utcnow() - recent_data.Tiempo[min_points_idx[-1]]).seconds} segundos.')
            lista_logs = [datetime.utcnow(),'V']
            df_logs.loc[len(df_logs)] = lista_logs
            print(min_points_idx[-1])
    except:
        pass
    
    if recent_data.Tiempo[-1:].item() >= datetime.strptime('2024-04-09 11:06:00','%Y-%m-%d %H:%M:%S'):
        break

Precio de compra en: 70752.01. Tiempo de compra: 2024-04-09 11:04:54.275000. Tiempo real de la señal: 2024-04-09 11:05:26.282946. Diferencia entre señal y compra: 32 segundos.
3999
Precio de venta en: 70769.3. Tiempo de venta: 2024-04-09 11:05:33.495000. Tiempo real de la señal: 2024-04-09 11:05:40.470443. Diferencia entre señal y compra: 6 segundos.
4038


In [20]:
df_logs

,Tiempo,Precio,Accion
0,2024-04-09 09:44:04.894922,0.0,C
1,2024-04-09 09:44:04.895918,0.0,V
2,2024-04-09 09:44:51.084390,0.0,V
3,2024-04-09 09:45:40.280867,0.0,C
4,2024-04-09 09:45:40.282868,0.0,V
5,2024-04-09 09:46:29.728494,0.0,V
6,2024-04-09 09:47:14.764556,0.0,C
7,2024-04-09 09:47:14.766558,0.0,V
8,2024-04-09 09:48:01.115127,0.0,V
9,2024-04-09 09:48:47.986176,0.0,C


In [25]:
lidx_buy = []
lidx_sell = []

for i in range(len(df_logs)-2):
    if df_logs.Accion[i] == 'C':
        idx_buy = recent_data[recent_data.Tiempo >= df_logs.Tiempo[i]].head(1).index.item()
        lidx_buy.append(idx_buy)
    elif df_logs.Accion[i] == 'V':
        idx_sell = recent_data[recent_data.Tiempo >= df_logs.Tiempo[i]].head(1).index.item()
        lidx_sell.append(idx_buy)

pmax_df = recent_data.iloc[lidx_buy].loc[:,['Tiempo','Precio']]
pmax_df['Accion'] = 'C'
pmin_df = recent_data.iloc[lidx_sell].loc[:,['Tiempo','Precio']]
pmin_df['Accion'] = 'V'
vc_df = pd.concat([pmax_df,pmin_df],axis=0)
vc_df = vc_df.reset_index()
vc_df = vc_df.set_index('index')
vc_df = vc_df.sort_index()
del_idx = []
vc_df = vc_df.reset_index()
for i in vc_df.index:
    try:
        if vc_df.Accion[i+1] == vc_df.Accion[i]:
            del_idx.append(i+1)
    except:
        pass
vc_df.drop(del_idx,inplace=True)
vc_df = vc_df.reset_index(drop=True)
vc_df['Diferencia_Precio_USDT'] = pd.Series(dtype='float')
for i in range(len(vc_df)-1):
    vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 1
profits = 0
for i in range(len(vc_df)-1):
    if vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
profits

C:\Users\tinma\AppData\Local\Temp\ipykernel_32736\592020137.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 1


351.9800000000105

In [24]:
pmax_df = recent_data.iloc[max_points_idx].loc[:,['Tiempo','Precio']]
pmax_df['Accion'] = 'C'
pmin_df = recent_data.iloc[min_points_idx].loc[:,['Tiempo','Precio']]
pmin_df['Accion'] = 'V'
vc_df = pd.concat([pmax_df,pmin_df],axis=0)
vc_df = vc_df.reset_index()
vc_df = vc_df.set_index('index')
vc_df = vc_df.sort_index()
del_idx = []
# vc_df = vc_df.reset_index()
for i in vc_df.index:
    try:
        if vc_df.Accion[i+1] == vc_df.Accion[i]:
            del_idx.append(i+1)
    except:
        pass
vc_df.drop(del_idx,inplace=True)
vc_df = vc_df.reset_index(drop=True)
vc_df['Diferencia_Precio_USDT'] = pd.Series(dtype='float')
for i in range(len(vc_df)-1):
    vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 1
profits = 0
for i in range(len(vc_df)-1):
    if vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
profits

C:\Users\tinma\AppData\Local\Temp\ipykernel_32736\1731861966.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 1


13024.959999999963

In [12]:
df_logs

,Tiempo,Precio,Accion
0,2024-04-09 07:12:22.803405,0.0,C
1,2024-04-09 07:12:22.804404,0.0,V
2,2024-04-09 07:12:30.787013,0.0,V
3,2024-04-09 07:12:38.701136,0.0,V
4,2024-04-09 07:12:46.590762,0.0,C
...,...,...,...
104,2024-04-09 07:34:21.677053,0.0,V
105,2024-04-09 07:34:43.170612,0.0,C
106,2024-04-09 07:35:08.492969,0.0,C
107,2024-04-09 07:35:35.658473,0.0,C


In [7]:
recent_data

,Tiempo,Par,Precio,Interpolacion,Primer_Derivada,PD_Interpolacion,Segunda_Derivada,SD_Interpolacion,Tercer_Derivada,TD_Interpolacion
0,2024-04-09 06:55:24.874,BTCUSDT,70763.99,70747.702261,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-09 06:55:25.742,BTCUSDT,70763.99,70749.704940,2.307235,NaN,NaN,NaN,NaN,NaN
2,2024-04-09 06:55:26.696,BTCUSDT,70752.66,70751.538993,1.922487,NaN,NaN,NaN,NaN,NaN
3,2024-04-09 06:55:27.909,BTCUSDT,70752.66,70753.207678,1.375668,NaN,NaN,NaN,NaN,NaN
4,2024-04-09 06:55:28.825,BTCUSDT,70752.66,70754.714256,1.644735,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2374,2024-04-09 07:35:12.091,BTCUSDT,70620.05,70614.537189,-0.712698,-0.783195,-0.065310,-0.070115,-0.004288,-0.004640
2375,2024-04-09 07:35:13.090,BTCUSDT,70620.00,70613.729034,-0.808964,-0.856848,-0.073727,-0.074928,-0.004818,-0.004878
2376,2024-04-09 07:35:14.070,BTCUSDT,70608.02,70612.868965,-0.877622,-0.934857,-0.079601,-0.080004,-0.005179,-0.005123
2377,2024-04-09 07:35:15.049,BTCUSDT,70605.07,70611.955624,-0.932932,-1.017350,-0.084262,-0.085351,-0.005461,-0.005374


In [34]:
vc_df

,index,Tiempo,Precio,Accion,Diferencia_Precio_USDT
0,1182,2024-04-09 06:19:52.632,71082.79,C,NaN
1,1182,2024-04-09 06:19:52.632,71082.79,V,0.00
2,1198,2024-04-09 06:20:09.097,71081.64,C,-1.15
3,1223,2024-04-09 06:20:34.170,71080.24,V,-1.40
4,1248,2024-04-09 06:20:59.546,71080.25,C,0.01
5,1256,2024-04-09 06:21:07.799,71060.01,V,-20.24
6,1256,2024-04-09 06:21:07.799,71060.01,C,0.00
7,1256,2024-04-09 06:21:07.799,71060.01,V,0.00
8,1309,2024-04-09 06:22:01.051,71050.72,C,-9.29
9,1356,2024-04-09 06:22:48.174,71054.87,V,4.15


In [24]:
recent_data

,Tiempo,Par,Precio,Interpolacion,Primer_Derivada,PD_Interpolacion,Segunda_Derivada,SD_Interpolacion,Tercer_Derivada,TD_Interpolacion
0,2024-04-09 05:59:50.291,BTCUSDT,71055.99,71060.447237,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-09 05:59:51.393,BTCUSDT,71055.99,71059.197628,-1.133946,NaN,NaN,NaN,NaN,NaN
2,2024-04-09 05:59:52.167,BTCUSDT,71055.99,71058.000466,-1.546720,NaN,NaN,NaN,NaN,NaN
3,2024-04-09 05:59:53.923,BTCUSDT,71055.99,71056.855733,-0.651898,NaN,NaN,NaN,NaN,NaN
4,2024-04-09 05:59:54.835,BTCUSDT,71055.99,71055.763409,-1.197724,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1369,2024-04-09 06:23:00.442,BTCUSDT,71051.33,71053.805679,3.157183,1.643262,0.215202,0.118518,-0.029273,-0.013897
1370,2024-04-09 06:23:01.892,BTCUSDT,71051.33,71055.237013,0.987127,1.727447,0.058059,0.104311,-0.009798,-0.015978
1371,2024-04-09 06:23:02.641,BTCUSDT,71051.33,71056.779297,2.059123,1.805065,0.103629,0.088093,-0.021653,-0.018130
1372,2024-04-09 06:23:04.001,BTCUSDT,71051.34,71058.435004,1.217432,1.875534,0.051815,0.069795,-0.013455,-0.020352


In [7]:
recent_data.loc[len(recent_data)-1]

Tiempo              2024-04-09 11:06:02.561000
Par                                    BTCUSDT
Precio                                 70760.0
Interpolacion                     70754.668798
Primer_Derivada                      -1.319554
PD_Interpolacion                     -1.423795
Segunda_Derivada                     -0.210366
SD_Interpolacion                      -0.21113
Tercer_Derivada                      -0.028484
TD_Interpolacion                     -0.025944
Name: 4067, dtype: object